# 1. Set up

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 2. Prepare data

In [45]:
df = pd.read_csv('matched_data.csv', nrows=30000)
df.head()

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,...,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,years,city,education
0,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,16.0,Советск (Калининградская область),Неоконченное высшее образование
1,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,полный день,Опыт работы 9 лет 4 месяца Системный админист...,"ООО ""Дилижанс""",Системный администратор,"Высшее образование 2011 МГУПИ ИТ/программист, ...",22.04.2019 13:51,Не указано,9.0,Санкт-Петербург,Высшее образование
2,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, сменный график, уд...",Опыт работы 3 года 10 месяцев Системный админ...,МАОУ СОШ №3,лаборант ИВТ,Неоконченное высшее образование 2020 НТСПИ Ест...,22.04.2019 06:05,Не указано,3.0,Нижний Тагил,Неоконченное высшее образование
3,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,полный день,Опыт работы 16 лет 2 месяца Сентябрь 2012 — п...,Государственная дума РФ,Помощник депутата,Высшее образование (Бакалавр) 2017 Московский ...,20.04.2019 16:15,Имеется собственный автомобиль,16.0,Москва,Высшее образование
4,48957743,Адверт Лайн,системный администратор,True,160000.0,NaN,От 3 до 6 лет,Удаленная работа,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",Привет! Компания Advert Line объединяет в себе...,...,"гибкий график, полный день, удаленная работа",Опыт работы 17 лет 3 месяца Декабрь 2017 — по...,"ООО ""Техкомплект""",Системный администратор,Высшее образование 1999 Красноярский государс...,10.04.2019 15:37,Имеется собственный автомобиль,17.0,Красноярск,Высшее образование


In [46]:
df.columns

Index(['Ids', 'Employer', 'Name', 'Salary', 'From', 'To', 'Experience',
       'Schedule', 'Keys', 'Description', 'Area', 'Professional roles',
       'Specializations', 'Profarea names', 'Published at', 'min_years',
       'index', 'Пол, возраст', 'ЗП', 'Ищет работу на должность:',
       'Город, переезд, командировки', 'Занятость', 'График', 'Опыт работы',
       'Последнее/нынешнее место работы', 'Последняя/нынешняя должность',
       'Образование и ВУЗ', 'Обновление резюме', 'Авто', 'years', 'city',
       'education'],
      dtype='object')

In [34]:
df.shape

(293931, 1)

In [47]:
df =df[['Name', 'Description', 'Keys', 'Experience', 'Ищет работу на должность:', 'Опыт работы', 'education']]
df.head()

,Name,Description,Keys,Experience,Ищет работу на должность:,Опыт работы,education
0,системный администратор,Привет! Компания Advert Line объединяет в себе...,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",От 3 до 6 лет,системный администратор,Опыт работы 16 лет 10 месяцев Август 2010 — п...,Неоконченное высшее образование
1,системный администратор,Привет! Компания Advert Line объединяет в себе...,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",От 3 до 6 лет,системный администратор,Опыт работы 9 лет 4 месяца Системный админист...,Высшее образование
2,системный администратор,Привет! Компания Advert Line объединяет в себе...,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",От 3 до 6 лет,системный администратор,Опыт работы 3 года 10 месяцев Системный админ...,Неоконченное высшее образование
3,системный администратор,Привет! Компания Advert Line объединяет в себе...,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",От 3 до 6 лет,системный администратор,Опыт работы 16 лет 2 месяца Сентябрь 2012 — п...,Высшее образование
4,системный администратор,Привет! Компания Advert Line объединяет в себе...,"['devops', 'Nagios', 'Nginx', 'PHP', 'Админист...",От 3 до 6 лет,системный администратор,Опыт работы 17 лет 3 месяца Декабрь 2017 — по...,Высшее образование


In [48]:
df['vacancy'] = df[['Name', 'Description', 'Keys', 'Experience']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df['resume'] = df[['Ищет работу на должность:', 'Опыт работы', 'education']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

df.drop(['Name', 'Description', 'Keys', 'Experience'], axis=1, inplace=True)
df.drop(['Ищет работу на должность:', 'Опыт работы', 'education'], axis=1, inplace=True)
print(df.head())

                                             vacancy  \
0  системный администратор Привет! Компания Adver...   
1  системный администратор Привет! Компания Adver...   
2  системный администратор Привет! Компания Adver...   
3  системный администратор Привет! Компания Adver...   
4  системный администратор Привет! Компания Adver...   

                                              resume  
0  системный администратор Опыт работы 16 лет 10 ...  
1  системный администратор Опыт работы 9 лет 4 ме...  
2  системный администратор Опыт работы 3 года 10 ...  
3  системный администратор Опыт работы 16 лет 2 м...  
4  системный администратор Опыт работы 17 лет 3 м...  


# 3. Tokenize data

In [49]:
X = df['vacancy']
y = df['resume']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [51]:
data = list(X_train)
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

# 4. Model initialization and vocabulary buiding

In [73]:
model = Doc2Vec(vector_size = 30,
min_count = 5,
epochs = 50,
alpha = 0.01
)

In [74]:
model.build_vocab(tagged_data)

keys = model.wv.key_to_index.keys()

print(len(keys))

12294


# 5. Train and save the model

In [75]:
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

# model.save('cv_job_maching.model')
# print("Model saved")

Training epoch 1/50
Training epoch 2/50
Training epoch 3/50
Training epoch 4/50
Training epoch 5/50
Training epoch 6/50
Training epoch 7/50
Training epoch 8/50
Training epoch 9/50
Training epoch 10/50
Training epoch 11/50
Training epoch 12/50
Training epoch 13/50
Training epoch 14/50
Training epoch 15/50
Training epoch 16/50
Training epoch 17/50
Training epoch 18/50
Training epoch 19/50
Training epoch 20/50
Training epoch 21/50
Training epoch 22/50
Training epoch 23/50
Training epoch 24/50
Training epoch 25/50
Training epoch 26/50
Training epoch 27/50
Training epoch 28/50
Training epoch 29/50
Training epoch 30/50
Training epoch 31/50
Training epoch 32/50
Training epoch 33/50
Training epoch 34/50
Training epoch 35/50
Training epoch 36/50
Training epoch 37/50
Training epoch 38/50
Training epoch 39/50
Training epoch 40/50
Training epoch 41/50
Training epoch 42/50
Training epoch 43/50
Training epoch 44/50
Training epoch 45/50
Training epoch 46/50
Training epoch 47/50
Training epoch 48/50
T

In [77]:
model.save('cv_job_maching_best.model')
print("Model saved")

Model saved


# 6. Model test

In [78]:
test_df = pd.concat([X_train, y_train], axis=1)
test_df.head()

,vacancy,resume
28465,"frontend-разработчик Aventica — агентство, кот...",frontend-разработчик Опыт работы 2 года 11 мес...
27622,программист 1с Обязанности: Поддержка типовых...,программист 1с Опыт работы 13 лет 3 месяца Де...
28376,html-верстальщик О проекте и компании: В Марке...,html-верстальщик Опыт работы 1 год 2 месяца H...
10917,руководитель проектов «Иннотех» — молодая гр...,руководитель проектов Опыт работы 14 лет 6 мес...
27234,программист 1с Российская компания ООО БУЛАТ с...,программист 1с Опыт работы 11 лет Программист...


In [79]:
def calculate_similarity(row):
    v1 = model.infer_vector(row['resume'].split())
    v2 = model.infer_vector(row['vacancy'].split())
    similarity = 100 * (np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
    return round(similarity, 2)

 
test_df['similarity_score'] = test_df.apply(calculate_similarity, axis=1)

test_df

,vacancy,resume,similarity_score
28465,"frontend-разработчик Aventica — агентство, кот...",frontend-разработчик Опыт работы 2 года 11 мес...,33.52
27622,программист 1с Обязанности: Поддержка типовых...,программист 1с Опыт работы 13 лет 3 месяца Де...,20.80
28376,html-верстальщик О проекте и компании: В Марке...,html-верстальщик Опыт работы 1 год 2 месяца H...,22.09
10917,руководитель проектов «Иннотех» — молодая гр...,руководитель проектов Опыт работы 14 лет 6 мес...,5.59
27234,программист 1с Российская компания ООО БУЛАТ с...,программист 1с Опыт работы 11 лет Программист...,57.70
...,...,...,...
29802,руководитель проекта Cloudfactory–компания раз...,руководитель проекта Опыт работы 11 лет Руков...,27.59
5390,php-разработчик KASSIR.RU - динамично развиваю...,php-разработчик Опыт работы 2 года PHP-разраб...,54.09
860,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 5 лет 3 ме...,37.11
15795,frontend-разработчик “Новоторика” - современна...,frontend-разработчик Опыт работы 1 год 4 месяц...,20.80


In [80]:
test_df.head(100)

,vacancy,resume,similarity_score
28465,"frontend-разработчик Aventica — агентство, кот...",frontend-разработчик Опыт работы 2 года 11 мес...,33.52
27622,программист 1с Обязанности: Поддержка типовых...,программист 1с Опыт работы 13 лет 3 месяца Де...,20.80
28376,html-верстальщик О проекте и компании: В Марке...,html-верстальщик Опыт работы 1 год 2 месяца H...,22.09
10917,руководитель проектов «Иннотех» — молодая гр...,руководитель проектов Опыт работы 14 лет 6 мес...,5.59
27234,программист 1с Российская компания ООО БУЛАТ с...,программист 1с Опыт работы 11 лет Программист...,57.70
...,...,...,...
22073,product manager Unlimint – это быстро развиваю...,product manager Опыт работы 20 лет 1 месяц KA...,26.60
14335,специалист технической поддержки Чем Вам предс...,специалист технической поддержки Опыт работы 7...,31.89
11274,руководитель проектов «Иннотех» — молодая гр...,руководитель проектов Опыт работы 6 лет 4 меся...,8.84
15726,frontend-разработчик “Новоторика” - современна...,frontend-разработчик Опыт работы 7 лет 6 месяц...,30.08


In [82]:
test_df.to_csv('results.csv', index=False)  

# App code

In [83]:
def calculate_similarity_new(row):
    v1 = model.infer_vector(resume_text.split())
    v2 = model.infer_vector(row.split())
    similarity = 100 * (np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
    return round(similarity, 2)

In [85]:
resume_text = 'системный администратор Опыт работы 16 лет 10 месяцев Неоконченное высшее образование'
tested = df
tested['similarity'] = df['vacancy'].apply(calculate_similarity_new)

In [90]:
tested['resume'] = resume_text
(tested.sort_values(by='similarity', ascending=False)).head(50)

,vacancy,resume,similarity
1445,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,79.22
3517,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,77.08
750,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,76.94
27413,php В связи с расширением штата интернет-магаз...,системный администратор Опыт работы 16 лет 10 ...,76.51
2727,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,76.09
949,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,75.97
3304,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,75.97
249,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,75.86
1837,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,75.82
2744,системный администратор Привет! Компания Adver...,системный администратор Опыт работы 16 лет 10 ...,75.74
